#Fine-Tuning NCBI BlueBERT for Named Entity Recognition


Bidirectional Encoder Representations from Transformers (BERT) is a method for training natural language processing (NLP) models published by Google in 2018 that has quickly established itself as the gold standard in a variety of common NLP tasks.

NCBI Bluebert represents a domain-specific version of BERT pretrained on a corpus of biomedical texts.

https://github.com/ncbi-nlp/ncbi_bluebert

In this notebook, we are replicating NCBI Bluebert's performance on tasks from the Biomedical Language Understanding Evaluation (BLUE) dataset, and taking a closer look at some of the predictions.

https://github.com/ncbi-nlp/BLUE_Benchmark


Installing/importing requirements.  

NCBI bluebert requires tensorflow 1.14 and is currently incompatible with later versions.  Note that prior to tensorflow 1.15, tensorflow and tensorflow-gpu are distinct packages.

In [0]:
!pip install tensorflow-gpu==1.14
import tensorflow as tf
print(tf.__version__)

!pip install transformers
!pip install wget

import transformers
import pandas as pd
import torch
import wget
import os
import numpy as np
import random

from zipfile import ZipFile
from google.colab import files
import glob

     |████████████████████████████████| 377.0MB 45kB/s 
     |████████████████████████████████| 491kB 28.0MB/s 
     |████████████████████████████████| 3.2MB 52.2MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0
     |████████████████████████████████| 450kB 5.6MB/s 
     |████████████████████████████████| 1.0MB 21.3MB/s 
     |████████████████████████████████| 870kB 43.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=89fa9b3b73005d36431cba6c3aeebeb5dd739f21e326ca1ea4af5e07e0cd8494
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=3498fae61ae709c3498917a2b3b32115e85b3dc29b01a3e7599cf507eadb05bf
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


Defining GPU device to use during training.


In [0]:
device_name = tf.test.gpu_device_name()

if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


Mounting personal Google drive for storing the model and training/testing data.

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

os.chdir('/content/gdrive/My Drive/Colab Notebooks/')

Clone NCBI_bluebert repository and download validation datasets.

In [0]:
url = 'https://github.com/ncbi-nlp/BLUE_Benchmark/releases/download/0.1/bert_data.zip'

if not os.path.exists('./ncbi_bluebert/'):
  !git clone https://github.com/ncbi-nlp/ncbi_bluebert.git

os.chdir('./ncbi_bluebert/')

if not os.path.exists('./bluebert_datasets.zip'):
  wget.download(url, './bluebert_datasets.zip')

if not os.path.exists('./bert_data/'):
  !unzip bluebert_datasets.zip

In [0]:
print("Contents of downloaded BC5CDR disease directory:")

print(os.listdir('./bert_data/BC5CDR/disease/'))

Contents of downloaded BC5CDR disease directory:
['train.tsv', 'devel.tsv', 'test.tsv']


Obtain NCBI BlueBERT pretrained weights, vocab, and config files.

This model was pretrained on uncased Pubmed corpora (4,000 M words).


In [0]:
url = 'https://ftp.ncbi.nlm.nih.gov/pub/lu/Suppl/NCBI-BERT/NCBI_BERT_pubmed_uncased_L-12_H-768_A-12.zip'

if not os.path.exists('./bluebert_model/'):
  os.mkdir('./bluebert_model')

if not os.path.exists('./NCBI_BERT_pubmed_uncased_L-12_H-768_A-12.zip'):
  wget.download(url, './NCBI_BERT_pubmed_uncased_L-12_H-768_A-12.zip')
  with ZipFile('NCBI_BERT_pubmed_uncased_L-12_H-768_A-12.zip', 'r') as f:
    f.extractall('./bluebert_model/')

print('Contents of downloaded pretrained weights directory:')
print(os.listdir('./bluebert_model/'))

Contents of downloaded pretrained weights directory:
['bert_config.json', 'bert_model.ckpt.data-00000-of-00001', 'bert_model.ckpt.index', 'bert_model.ckpt.meta', 'checkpoint', 'vocab.txt']


In [0]:
"""
Fine-tuning NCBI_bluebert pretrained model on bc5cdr disease dataset.
"""

%run bluebert/run_bluebert_ner.py \
 --task_name='bc5cdr' \
 --do_prepare=true \
 --do_train=true \
 --do_eval=true \
 --do_predict=true  \
 --vocab_file=bluebert_model/vocab.txt \
 --bert_config_file=bluebert_model/bert_config.json \
 --init_checkpoint=bluebert_model/bert_model.ckpt \
 --data_dir=bert_data/BC5CDR/disease/ \
 --num_train_epochs=3.0 \
 --do_lower_case=False \
 --output_dir=ft_output

/usr/local/lib/python3.6/dist-packages/absl/flags/_validators.py:359: UserWarning: Flag --task_name has a non-None default value; therefore, mark_flag_as_required will pass even if flag is not specified in the command line!
  'command line!' % flag_name)
W0112 23:29:51.031541 140028365911936 deprecation_wrapper.py:119] From /content/gdrive/My Drive/Colab Notebooks/ncbi_bluebert/bluebert/run_bluebert_ner.py:652: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.



W0112 23:29:51.038688 140028365911936 deprecation_wrapper.py:119] From /content/gdrive/My Drive/Colab Notebooks/ncbi_bluebert/bluebert/run_bluebert_ner.py:652: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.



W0112 23:29:51.044361 140028365911936 deprecation_wrapper.py:119] From /content/gdrive/My Drive/Colab Notebooks/ncbi_bluebert/bert/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



W0112 23:29:51.237095 140028365911936 deprecation_wrapper.py:119] From /content/gdrive/My Drive/Colab Notebooks/ncbi_bluebert/bluebert/run_bluebert_ner.py:672: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

I0112 23:29:51.795561 140028365911936 utils.py:141] NumExpr defaulting to 2 threads.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



...

INFO:tensorflow:prediction_loop marked as finished


I0112 23:43:36.840556 140028365911936 error_handling.py:96] prediction_loop marked as finished


{1: 'B', 2: 'I', 3: 'O', 4: 'X', 5: '[CLS]', 6: '[SEP]'}
ft_output/test_labels.txt
ft_output/test_labels_errs.txt
INFO:tensorflow:Reading: ft_output/test_labels.txt


I0112 23:43:37.477514 140028365911936 run_bluebert_ner.py:814] Reading: ft_output/test_labels.txt


processed 124669 tokens with 4424 phrases; found: 4514 phrases; correct: 3723.
accuracy:  98.45%; precision:  82.48%; recall:  84.15%; FB1:  83.31
                 : precision:  82.48%; recall:  84.15%; FB1:  83.31  4514



Evaluating the fine-tuned model's predictions on the test data.

The paper reports an F-score of 86.6% using this model, fine-tuned for this task.  The difference is probably in the number of epochs: 3 vs. 20.


In [0]:
with open('ft_output/test_results.txt') as f:
  for line in f.readlines():
    print(line)

eval_f = 0.8391248

eval_precision = 0.8379883

eval_recall = 0.84028316

global_step = 1230

loss = 11.395871



I'm interested in taking a look at some of the 'misses' our fine-tuned data made with the test data:

False Positive:  Model incorrectly predicted a token to be part of a named entity.

False Negative: Model incorrectly predicted a token to not be part of a named entity.

One of BERT's output files, test_labels.txt, includes each test example tokenized with its correct label and predicted label:

<word> <label> <predicted label>


The test_labels file is split into text blocks, which are each assigned a number.  These are then written into a tsv file which can be read into a pandas dataframe.

In [0]:
"""
test_labels.txt format:
<word> <label> <predicted label>

BIO token labels:
B - first word in named entity
I - later word in named entity
O - not a member of a named entity
"""

def parse_test_labels_text_to_tsv(test_labels_file, out_tsv):

  with open(test_labels_file) as f:
    text_blocks = f.read().split('\n\n')

  idx = 0
  text_block_dict = {}

  with open(out_tsv, 'w') as out_tsv_h:
    out_tsv_h.write('text_block\ttoken\tlabel\tprediction\n')
    for text_block in text_blocks:
      idx += 1
      text_block_lines = text_block.split('\n')
      for line in text_block_lines:
        split_line = line.split(' ')
        if len(split_line) == 3:  
          out_tsv_h.write('%d\t%s\t%s\t%s\n' % (idx, split_line[0], split_line[1], split_line[2]))

test_labels_in = 'ft_output/test_labels.txt'
test_labels_tsv = 'ft_output/test_labels.tsv'

parse_test_labels_text_to_tsv(test_labels_in, test_labels_tsv)

In [0]:
test_labels_df = pd.read_csv(test_labels_tsv, sep='\t', header=0)
error_test_labels_df = test_labels_df.loc[test_labels_df['label'] != test_labels_df['prediction']]

In [0]:
#False negatives
fn_text_blocks = set(error_test_labels_df.loc[error_test_labels_df.prediction == 'O'].text_block.tolist())
print("There are %d text blocks with false negatives." % len(fn_text_blocks))

#False positives
fp_text_blocks = set(error_test_labels_df.loc[error_test_labels_df.label == 'O'].text_block.tolist())
print("There are %d text blocks with false positives." % len(fp_text_blocks))

#Both
text_blocks_with_both_errors = set.intersection(fn_text_blocks, fp_text_blocks)
print("There are %d text blocks with both a false negative and false positive prediction." % len(text_blocks_with_both_errors))

#Neither
all_text_blocks = set(test_labels_df.text_block.to_list())
text_blocks_with_no_errors = all_text_blocks - set.union(fn_text_blocks, fp_text_blocks)
print("There are %d text blocks with neither error type." % len(text_blocks_with_no_errors))

There are 451 text blocks with false negatives.
There are 514 text blocks with false positives.
There are 37 text blocks with both a false negative and false positive prediction.
There are 6064 text blocks with neither error type.


# Example Errors

In [0]:
def print_text_block(test_labels_df, text_block_idx):
  print(text_block_idx)
  sample_df = test_labels_df.loc[test_labels_df.text_block == text_block_idx]
  sample_text_tokens = sample_df.token.tolist()
  sample_text = '%s' % sample_text_tokens[0]

  #Convert list of tokens into a readable, left-to-right sentence.
  for token in sample_text_tokens[1:]:
    if token.isalnum():
      sample_text += ' %s' % token
    else:
      sample_text += '%s' % token

  #Extract all tokens either labeled or predicted to be entities
  error_df = sample_df[(sample_df.label != 'O') | (sample_df.prediction != 'O')][['token', 'label', 'prediction']]
  print(sample_text)
  print(error_df)

In [0]:
#False negative examples
fn_samples = random.sample(fn_text_blocks, 10)

for fn_sample in fn_samples:
  print_text_block(test_labels_df, fn_sample)
  print('\n')

2508
During hypotension, the heart rate was stable without tachy- or bradycardia.
             token label prediction
44715  hypotension     B          B
44723        tachy     B          O
44724            -     I          O
44725           or     I          O
44726  bradycardia     I          B


4277
Sulfadiazine acute nephrotoxicity is reviving specially because of its use in toxoplasmosis in HIV- positive patients.
                token label prediction
77334           acute     B          O
77335  nephrotoxicity     I          B
77344   toxoplasmosis     B          B


5325
Specific gravity was greater in the 15/ HTN group( 1. 043+/- 0. 002) versus the 90/ HTN( 1. 036
      token label prediction
95673   HTN     B          O
95690   HTN     B          O


5479
HT is not indicated for the routine management of chronic disease.
         token label prediction
98048  chronic     B          O
98049  disease     I          O


4971
After one year of electrical stimulation of the STN, 

In [0]:
#False positive examples
fp_samples = random.sample(fp_text_blocks, 10)

for fp_sample in fp_samples:
  print_text_block(test_labels_df, fp_sample)
  print('\n')

1865
" This report describes subtotal infarction of the kidney due to thrombosis of a normal renal artery.
            token label prediction
33029  infarction     B          O
33030          of     O          B
33031         the     O          I
33032      kidney     O          I
33033         due     O          I
33035  thrombosis     B          O
33036          of     I          B
33037           a     I          O
33038      normal     I          O
33039       renal     I          O
33040      artery     I          O


1906
By means of a computer program, periods and amplitudes of tremor oscillations were calculated and classified.
              token label prediction
33758        tremor     B          B
33759  oscillations     O          I


6987
) rat model of sAD, tested by Morris Water Maze and Passive Avoidance test, respectively.
       token label prediction
124531   sAD     O          B


3283
Hypotension induced by ATP was accompanied by significant decreases in mean pulmo

#Relation Extraction


In [0]:
"""
DDI extraction 2013 corpus is a collection of 792 texts selected from the DrugBank database and other 233 Medline abstracts. 
In our benchmark, we use 624 train files and 191 test files to evaluate the performance and report the macro-average F1-score of the four DDI types.
"""

'\nDDI extraction 2013 corpus is a collection of 792 texts selected from the DrugBank database and other 233 Medline abstracts In our benchmark, we use 624 train files and 191 test files to evaluate the performance and report the macro-average F1-score of the four DDI types.\n'

In [0]:
%run bluebert/run_bluebert.py \
  --do_train=true \
  --do_eval=true \
  --do_predict=true \
  --task_name="ddi" \
  --vocab_file=bluebert_model/vocab.txt \
  --bert_config_file=bluebert_model/bert_config.json \
  --init_checkpoint=bluebert_model/bert_model.ckpt \
  --num_train_epochs=10.0 \
  --data_dir=bert_data/ddi2013-type/ \
  --output_dir=re_ft_output \
  --do_lower_case=False

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#Scoring predictions
with open('ft_output/eval_results.txt') as f:
  print(f.readlines())

eval_f = 0.97956157

eval_precision = 0.9787742

eval_recall = 0.98035544

global_step = 1230

loss = 0.9638384



In [0]:
test_data_df = pd.read_csv('bert_data/ddi2013-type/test.tsv', sep='\t')
test_results_df = pd.read_csv('re_ft_output/test_results.tsv', sep='\t', header=None)

mapped_results_df = pd.DataFrame({'index': test_data_df['index'], 
                                 'text': test_data_df['sentence'],
                                 'label': test_results_df.idxmax(axis=1)})

mapped_results_df

,index,text,label
0,DDI-DrugBank.d572.s0.p0,@DRUG$: Concomitant @DRUG$ administration decr...,4
1,DDI-DrugBank.d572.s0.p1,@DRUG$: Concomitant cholestyramine administrat...,4
2,DDI-DrugBank.d572.s0.p2,Cholestyramine: Concomitant @DRUG$ administrat...,3
3,DDI-DrugBank.d572.s1.p0,The incremental LDL-C reduction due to adding ...,1
4,DDI-DrugBank.d572.s10.p0,Patients who take both @DRUG$ and @DRUG$ shoul...,0
...,...,...,...
5756,DrugDDI.d21928724.s6.p6,The effect of curcumin (100 mg/kg) was similar...,4
5757,DrugDDI.d21928724.s6.p7,The effect of curcumin (100 mg/kg) was similar...,4
5758,DrugDDI.d21928724.s6.p8,The effect of curcumin (100 mg/kg) was similar...,4
5759,DrugDDI.d21928724.s6.p9,The effect of curcumin (100 mg/kg) was similar...,4


In [0]:
test_data_df

,index,sentence,label
0,DDI-DrugBank.d572.s0.p0,@DRUG$: Concomitant @DRUG$ administration decr...,DDI-false
1,DDI-DrugBank.d572.s0.p1,@DRUG$: Concomitant cholestyramine administrat...,DDI-false
2,DDI-DrugBank.d572.s0.p2,Cholestyramine: Concomitant @DRUG$ administrat...,DDI-mechanism
3,DDI-DrugBank.d572.s1.p0,The incremental LDL-C reduction due to adding ...,DDI-effect
4,DDI-DrugBank.d572.s10.p0,Patients who take both @DRUG$ and @DRUG$ shoul...,DDI-advise
...,...,...,...
5756,DrugDDI.d21928724.s6.p6,The effect of curcumin (100 mg/kg) was similar...,DDI-false
5757,DrugDDI.d21928724.s6.p7,The effect of curcumin (100 mg/kg) was similar...,DDI-false
5758,DrugDDI.d21928724.s6.p8,The effect of curcumin (100 mg/kg) was similar...,DDI-false
5759,DrugDDI.d21928724.s6.p9,The effect of curcumin (100 mg/kg) was similar...,DDI-false
